# STEP 1: Import FaaSET

In [ ]:
import FaaSET

# STEP 2: Create a dummy function. This will setup the folder that we will add files to in the next cells...

In [ ]:
@FaaSET.cloud_function(platform="AWS Docker Debian")
def hello_world_c(request, context): 
    return {"message": "Dummy Function"}   

hello_world_c({}, None)

# STEP 3: Create the C file:

In [29]:
%%writefile functions/hello_world_c/hello_world.c

#include <stdio.h>

int main()
{
    printf("Dello");
    return 0;
}

Overwriting functions/hello_world_c/hello_world.c


# STEP 4: Update the Dockerfile to compile the C:

In [27]:
%%writefile functions/hello_world_c/Dockerfile

#FROM python:buster as build-image
FROM python:3.8-slim-buster

# Update and install curl. 
RUN apt update
RUN apt install -y curl build-essential

# Define and create function directory
ENV FUNCTION_DIR="/var/task"
RUN mkdir -p ${FUNCTION_DIR}

# Copy function code
COPY . ${FUNCTION_DIR}

RUN pip install boto3 botocore

# Install the runtime interface client
RUN pip install \
        --target ${FUNCTION_DIR} \
        awslambdaric

# Install the runtime interface emulator
RUN curl -Lo /usr/local/bin/aws-lambda-rie https://github.com/aws/aws-lambda-runtime-interface-emulator/releases/latest/download/aws-lambda-rie
RUN chmod +x /usr/local/bin/aws-lambda-rie

# Set working directory
WORKDIR ${FUNCTION_DIR}

# Compile C code
RUN gcc -o hello_world hello_world.c

# Set entry script that determines whether to run RIE if running locally
COPY ./entry_script.sh /entry_script.sh
RUN chmod +x /entry_script.sh
ENTRYPOINT [ "/entry_script.sh" ]

# Set handler
CMD [ "lambda_function.lambda_handler" ]

Overwriting functions/hello_world_c/Dockerfile


# Step 5: Create your real function that will shell out and run the C:

In [26]:
@FaaSET.cloud_function(platform="AWS Docker Debian", config={"memory": 259, "profile": "personal"})
def hello_world_c(request, context): 
    import subprocess
    command = "./hello_world"
    output = subprocess.check_output(command.split()).decode('ascii')
    return {"message": output}   

hello_world_c({}, None)

{'message': 'Dello'}